In [ ]:
import torch

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from src.model.pointwise.model import HomophilyContentCNN

In [ ]:
import hiddenlayer as hl
import tensorwatch as tw

In [ ]:
import numpy as np

In [ ]:
from src.constants import MAX_LENGTH_USER_REPRESENATION, MAX_LENGTH_COMMENT_SECTION
from src.model.pointwise.model import HomophilyContentCNN
from src.utils.utils import create_node2vec_embedding_layer, save_config
from src.dataloader.pointwise.utils import get_data_loader

In [ ]:
user_embedding_path = ''

In [ ]:
dropout=0.5
user_num_kernels=50
# number of kernels
section_num_kernels=50
user_kernel_size=5  # number of words in window
section_kernel_size=5
user_latent_factors=50  # embedding size
section_latent_factors=50
freeze_embeddings=False

In [ ]:
NODE2VEC_EMB_DIM, num_authors, node2vec_emb_layer, author_to_pos_dict = create_node2vec_embedding_layer(
        user_embedding_path, True)



In [ ]:
model = DeepCoNN(
            MAX_LENGTH_USER_REPRESENATION,
            MAX_LENGTH_COMMENT_SECTION,
            dropout=dropout,
            user_num_kernels=user_num_kernels,
            # number of kernels
            section_num_kernels=section_num_kernels,
            user_kernel_size=user_kernel_size,  # number of words in window
            section_kernel_size=section_kernel_size,
            user_latent_factors1=user_latent_factors,  # embedding size
            section_latent_factors1=section_latent_factors,
            freeze_embeddings=freeze_embeddings
)

In [ ]:
train_loader, _ = get_data_loader(num_authors, author_to_pos_dict, 1, True, 2,
                                               None)

In [ ]:
input_batch = next(iter(train_loader))

In [ ]:
from train_deepconn import transform_fn

In [ ]:
transforms = [
    # Fold Conv, BN, RELU layers into one
    hl.transforms.Fold("Gather > Cast", "Lookup", "Lookup Word"),
    hl.transforms.Prune("Constant"),
    hl.transforms.Fold("Squeeze > Gather", 'step1'),
    hl.transforms.Fold("step1 > ATen > Cast", 'Look', "Lookup User"),
    hl.transforms.Prune("step1"),
    hl.transforms.Fold("Sigmoid > Squeeze", 'out', 'Sigmoid'),
    hl.transforms.Fold("Conv > Relu > Squeeze", 'Convolution', 'Convolution'),
    hl.transforms.Fold("MaxPool > Squeeze", "Max-pooling", 'Max-pooling'),
    hl.transforms.Fold("Linear > Relu > Dropout", 'linear'),
    # Fold repeated blocks
    hl.transforms.FoldDuplicates(),
]

In [ ]:
model.cuda()

In [ ]:
graph = hl.build_graph(model, transform_fn(input_batch)[0], transforms=transforms)

In [ ]:
graph

In [ ]:
graph.format = 'pdf'

In [ ]:
graph.save('visualizations/deepconn.pdf')